# Module 11 Challenge
## Deliverable 2: Scrape and Analyse Mars Weather Data

### Step 0: Dependencies and Setup

In [ ]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html

url = 'https://static.bc-edx.com/data/web/mars_facts/temperature.html'
browser.visit(url)


### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
# Retrieve the HTML content of the page
html = browser.html

# Create a Beautiful Soup object
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Extract all rows of data
table = soup.find('table')

# Extract all rows of data
rows = table.find_all('tr')

# Print the rows of data
for row in rows:
    print(row)

In [ ]:
# Print the rows using the prettify() method
for row in rows:
    print(row.prettify())

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
data = []

# Loop through the scraped data to create a list of rows
for row in rows[1:]: # Skipping the header row
    # Find all columns in each row
    cols = row.find_all('td')

    # Extract the text from each column and strip the whitespace
    row_data = [col.text.strip() for col in cols]

    # Append the list of row data to the list of rows
    data.append(row_data)

# Print the data to check that the data was scraped correctly
for row in data:
    print(row)


In [ ]:
# Stripping the header row of the table
header_row = rows[0]
column_names = [col.text.strip() for col in header_row.find_all('th')]

column_names

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
df = pd.DataFrame(data, columns=column_names)

df.head()

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
print(df.dtypes)

In [ ]:
# Change data types for data analysis
df['terrestrial_date'] = pd.to_datetime(df['terrestrial_date'])

# Convert 'sol', 'ls' and 'month' columns to integers
df['sol'] = df['sol'].astype(int)
df['ls'] = df['ls'].astype(int)
df['month'] = df['month'].astype(int)

# Convert 'pressure' and 'min_temp' columns to floats
df['pressure'] = df['pressure'].astype(float)
df['min_temp'] = df['min_temp'].astype(float)

df.dtypes


### Step 5: Analyse the Data

Analyse your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
# Determine the number of unique months on Mars
unique_months = df['month'].nunique()

# Print the number of unique months
print(f"There are {unique_months} unique months on Mars.")

In [ ]:
# Determine the number of unique Martian days (sols) in the dataset
unique_id = df['id'].nunique()

# Print the number of unique Martian days
print(f"There are {unique_id} Martian days' worth of data in the dataset.")

In [ ]:
# 3. What is the average low temperature by month?
# The average minimum daily temperature for each month
avg_min_temp_by_month = df.groupby('month')['min_temp'].mean()

# Print the results
print(avg_min_temp_by_month)

In [ ]:
# Plot the average minimum daily temperature for each month
plt.figure(figsize=(10, 6))
avg_min_temp_by_month.plot(kind='bar')
plt.xlabel('Month')
plt.ylabel('Average Minimum Temperature (Celsius)')
plt.title('Average Minimum Daily Temperature by Month on Mars')
plt.xticks(rotation=0)

# Save the plot as a .png file
plt.savefig('outputs/average_min_temp_by_month.png', bbox_inches='tight')

plt.show()

In [ ]:
# Save the DataFrame to a CSV file in the outputs folder
df.to_csv('Outputs/mars_weather_data.csv', index=False)

In [ ]:
# Sort the values
sorted_avg_min_temp_by_month = avg_min_temp_by_month.sort_values()

# Plot the average minimum daily temperature for each month
plt.figure(figsize=(10, 6))
bars = plt.bar(sorted_avg_min_temp_by_month.index, sorted_avg_min_temp_by_month.values, color='grey')

# Color the lowest temperature bar blue and the highest temperature bar red
bars[0].set_color('blue')
bars[-1].set_color('red')

plt.xlabel('Month')
plt.ylabel('Average Minimum Temperature (Celsius)')
plt.title('Average Minimum Daily Temperature by Month on Mars')
plt.xticks(ticks=sorted_avg_min_temp_by_month.index, labels=sorted_avg_min_temp_by_month.index)

plt.show

In [ ]:
# Find the month with the lowest average minimum temperature
coldest_month = avg_min_temp_by_month.idxmin()
coldest_temp = avg_min_temp_by_month.min()

# Find the month with the highest average minimum temperature
warmest_month = avg_min_temp_by_month.idxmax()
warmest_temp = avg_min_temp_by_month.max()

# Print the results
print(f"The coldest month on Mars at Curiosity's location is Month {coldest_month} with an average minimum temperature of {coldest_temp:.2f}°C.")
print(f"The warmest month on Mars at Curiosity's location is Month {warmest_month} with an average minimum temperature of {warmest_temp:.2f}°C.")

In [ ]:
# Calculate the average minimum daily temperature for each month
avg_min_temp_by_month = df.groupby('month')['min_temp'].mean()

# Sort the values
sorted_avg_min_temp_by_month = avg_min_temp_by_month.sort_values()

# Plot the average minimum daily temperature for each month
plt.figure(figsize=(10, 6))
bars = plt.bar(sorted_avg_min_temp_by_month.index, sorted_avg_min_temp_by_month.values, color='grey')

# Color the lowest temperature bar blue and the highest temperature bar red
bars[0].set_color('blue')
bars[-1].set_color('red')

plt.xlabel('Month')
plt.ylabel('Average Minimum Temperature (Celsius)')
plt.title('Average Minimum Daily Temperature by Month on Mars')
plt.xticks(ticks=sorted_avg_min_temp_by_month.index, labels=sorted_avg_min_temp_by_month.index)  # Ensure all months are shown

# Add text for the coldest and warmest months
coldest_month = sorted_avg_min_temp_by_month.idxmin()
coldest_temp = sorted_avg_min_temp_by_month.min()
warmest_month = sorted_avg_min_temp_by_month.idxmax()
warmest_temp = sorted_avg_min_temp_by_month.max()

text = (
    f"The coldest month on Mars at Curiosity's location is Month {coldest_month} with an average minimum temperature of {coldest_temp:.2f}°C.\n"
    f"The warmest month on Mars at Curiosity's location is Month {warmest_month} with an average minimum temperature of {warmest_temp:.2f}°C."
)
plt.gcf().text(0.5, -0.15, text, ha='center', fontsize=10, wrap=True)

# Adjust the layout to make room for the text
plt.tight_layout(rect=[0, 0.1, 1, 0.95])

# Save the plot with the text to the outputs folder as a .png file
plt.savefig('outputs/average_min_temp_by_month_with_text.png', bbox_inches='tight')

# Display the plot
plt.show()


In [ ]:
# 4. What is the average pressure by Martian month?
# The average pressure for each month
avg_pressure_by_month = df.groupby('month')['pressure'].mean()

# Print the results
print(avg_pressure_by_month)

In [ ]:
# Calculate the average atmospheric pressure for each month
avg_pressure_by_month = df.groupby('month')['pressure'].mean()

# Sort the values
sorted_avg_pressure_by_month = avg_pressure_by_month.sort_values()

# Plot the average atmospheric pressure for each month
plt.figure(figsize=(10, 6))
bars = plt.bar(sorted_avg_pressure_by_month.index, sorted_avg_pressure_by_month.values, color='grey')

# Color the lowest pressure bar blue and the highest pressure bar red
bars[0].set_color('blue')
bars[-1].set_color('red')

plt.xlabel('Month')
plt.ylabel('Average Atmospheric Pressure (Pa)')
plt.title('Average Atmospheric Pressure by Month on Mars')
plt.xticks(ticks=sorted_avg_pressure_by_month.index, labels=sorted_avg_pressure_by_month.index)  # Ensure all months are shown

# Add text for the lowest and highest pressure months
lowest_pressure_month = sorted_avg_pressure_by_month.idxmin()
lowest_pressure_value = sorted_avg_pressure_by_month.min()
highest_pressure_month = sorted_avg_pressure_by_month.idxmax()
highest_pressure_value = sorted_avg_pressure_by_month.max()

text = (
    f"The month with the lowest average atmospheric pressure is Month {lowest_pressure_month} with {lowest_pressure_value:.2f} Pa.\n"
    f"The month with the highest average atmospheric pressure is Month {highest_pressure_month} with {highest_pressure_value:.2f} Pa."
)
plt.gcf().text(0.5, -0.15, text, ha='center', fontsize=10, wrap=True)

# Adjust the layout to make room for the text
plt.tight_layout(rect=[0, 0.1, 1, 0.95])

# Save the plot with the text to the outputs folder as a .png file
plt.savefig('outputs/average_pressure_by_month_with_text.png', bbox_inches='tight')

# Display the plot
plt.show()


In [ ]:
# Plot the minimum temperature as a time series
plt.figure(figsize=(12, 6))
plt.plot(df['terrestrial_date'], df['min_temp'], color='blue', linewidth=1)

plt.xlabel('Terrestrial Date')
plt.ylabel('Minimum Temperature (Celsius)')
plt.title('Minimum Temperature on Mars Over Time')
plt.grid(True)

# Save the plot to the outputs folder as a .png file
plt.savefig('outputs/min_temp_time_series.png', bbox_inches='tight')

# Display the plot
plt.show()

In [ ]:
# Plot the solar longitude (ls) as a time series
plt.figure(figsize=(12, 6))
plt.plot(df['terrestrial_date'], df['ls'], color='purple', linewidth=1)

plt.xlabel('Terrestrial Date')
plt.ylabel('ls')
plt.title('Ls Over Time')
plt.grid(True)

# # Save the plot to the outputs folder as a .png file
# plt.savefig('outputs/min_temp_time_series.png', bbox_inches='tight')

# Display the plot
plt.show()

In [ ]:
# Ensure the 'terrestrial_date' column is in datetime format
df['terrestrial_date'] = pd.to_datetime(df['terrestrial_date'])

# Sort the DataFrame by 'sol' to ensure chronological order in Martian days
df = df.sort_values(by='sol')

# Identify all entries where 'ls' is 0 (beginning of the Martian year)
ls_zero_df = df[df['ls'] == 0][['terrestrial_date', 'ls']]

# Print the DataFrame
print(ls_zero_df)

# Save the DataFrame to a CSV file in the outputs folder
ls_zero_df.to_csv('outputs/ls_zero_dates.csv', index=False)

In [ ]:
# Ensure the 'terrestrial_date' column is in datetime format
df['terrestrial_date'] = pd.to_datetime(df['terrestrial_date'])

# Sort the DataFrame by 'sol' to ensure chronological order in Martian days
df = df.sort_values(by='sol')

# Identify all entries where 'ls' is 0 (beginning of the Martian year)
ls_zero_df = df[df['ls'] == 0][['terrestrial_date', 'ls']]

# Calculate the difference in days between each date entry and the one above
ls_zero_df['days_diff'] = ls_zero_df['terrestrial_date'].diff().dt.days

# Print the DataFrame
print(ls_zero_df)

# Save the DataFrame to a CSV file in the outputs folder
ls_zero_df.to_csv('outputs/ls_zero_dates_with_diff.csv', index=False)

In [ ]:
earth_days_per_martian_year = int(ls_zero_df['days_diff'].max())
earth_days_per_martian_year

print(f"There are {earth_days_per_martian_year} terrestrial (Earth) days in a Martian year.")

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
# Done with each bit of code above; please refer to the Outputs folder for the CSV and PNG files.

In [ ]:
browser.quit()